# Using Shared　Library with Python

The advantage of Python is that it is flexible and easy to program. The time it takes to setup a new calulation is therefore short. But for certain types of calculations Python (and any other interpreted language) can be very slow. It is particularly iterations over large arrays that is difficult to do efficiently.

Such calculations may be implemented in a compiled language such as C or Fortran. In Python it is relatively easy to call out to libraries with compiled C or Fortran code. 

But before we go ahead and work on optimizing anything, it is always worthwhile to ask.... 

In [1]:
import seuif97
%timeit seuif97.pt(15,535,4)

The slowest run took 449.06 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.52 µs per loop


In [2]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

1000 loops, best of 3: 637 µs per loop


The **slowest** run took xxx.xx times **longer** than the **fastest**. This could mean that an **intermediate result** is being **cached.**

* Algorithm of the High-speed IAPWS-IF97 Library: 

   * 王培红,贾俊颖,程懋华. 水和水蒸汽热力性质IAPWS_IF97公式的通用计算模型[J]. 动力工程. 2001 21(6)：1564-1567(EI)


## ctypes

ctypes is a Python library for calling out to C code. We manually need to load the library and set properties such as the functions return and argument types. On the otherhand we do not need to touch the C code at all. 

In [6]:
%%file functions.c

#include <stdio.h>

void hello(int n);
double dprod(double *x, int n);
void dcumsum(double *a, double *b, int n);

void hello(int n)
{
    int i;
    for (i = 0; i < n; i++)
    {
        printf("C says hello\n");
    }
}

double dprod(double *x, int n)
{
    double y = 1.0;
    for (int i = 0; i < n; i++)
    {
        y *= x[i];
    }
    return y;
}

void dcumsum(double *a, double *b, int n)
{
    b[0] = a[0];
    for (int i = 1; i < n; i++)
    {
        b[i] = a[i] + b[i-1];
    }
}

Overwriting functions.c


Compile the C file into a shared library:

In [7]:
!gcc -c -O3 -Wall -fPIC -o functions.o functions.c
!gcc -o libfunctions.dll -shared functions.o

The result is a compiled shared library `libfunctions.dll`

Now we need to write wrapper functions to access the C library: 
To load the library we use the **ctypes** package, which included in the Python standard library (with extensions from numpy for passing arrays to C). Then we manually set the types of the argument and return values (no automatic code inspection here!). 

In [3]:
%%file functions.py

from ctypes import c_int, c_double,c_void_p
import numpy

_libfunctions = numpy.ctypeslib.load_library('libfunctions', '.')

_libfunctions.hello.argtypes = [c_int]
_libfunctions.hello.restype  =  c_void_p

_libfunctions.dprod.argtypes = [numpy.ctypeslib.ndpointer(dtype=numpy.float), c_int]
_libfunctions.dprod.restype  = c_double

_libfunctions.dcumsum.argtypes = [numpy.ctypeslib.ndpointer(dtype=numpy.float), numpy.ctypeslib.ndpointer(dtype=numpy.float),c_int]
_libfunctions.dcumsum.restype  = c_void_p

def hello(n):
    return _libfunctions.hello(int(n))

def dprod(x, n=None):
    if n is None:
        n = len(x)
    x = numpy.asarray(x, dtype=numpy.float)
    return _libfunctions.dprod(x, int(n))

def dcumsum(a, n):
    a = numpy.asarray(a, dtype=numpy.float)
    b = numpy.empty(len(a), dtype=numpy.float)
    _libfunctions.dcumsum(a, b, int(n))
    return b

Overwriting functions.py


In [4]:
%%file run_hello_c.py

import functions
functions.hello(3)

Overwriting run_hello_c.py


In [5]:
!python run_hello_c.py

C says hello
C says hello
C says hello


### Product function:

In [ ]:
import functions
functions.dprod([1,2,3,4,5]) 

### Further reading

* http://docs.python.org/3/library/ctypes.html
* http://www.scipy.org/Cookbook/Ctypes